In [ ]:
# PyGame template.
 
# Import standard modules.
import numpy as np
import random
from collections import deque
from tensorflow.keras.layers import Dense,Input,Multiply
from tensorflow.keras.models import Model
import tensorflow as tf


actions = [(-1,0),(1,0),(-1,1),(1,1)]
batch_size = 64
state_shape = 13
action_size = 4
learning_rate =  0.00025
hidden_neurons = 512
random_decay = 0.01
memory_gamma = 0.9
WIZARD_HEIGHT = 103

def step(bullets,enemy_spawner,wizard,action,state):
    done = False
    reward = 0
    if (wizard.y - vel > 0) and (action[0]==1) :
        wizard.y -= action[0]*vel
    if (wizard.y + WIZARD_HEIGHT +vel<height) and (action[0]==-1):
        wizard.y -= action[0]*vel
    state[0]=wizard.y
    for enemy in enemy_spawner.enemy_group:
        if wizard.colliderect(enemy):
          reward -= 200
          done = True
          break  
    if(not done):
        reward += 1
    for bullet in bullets:    
        for enemy in enemy_spawner.enemy_group:
          if bullet.colliderect(enemy):
            reward +=200
    if (action[1]==1) and len(bullets)<MAX_BULLET :
        bullet= pygame.Rect(wizard.x + wizard.width , wizard.y + wizard.height//2 , 10 , 5)
        bullets.append(bullet)
        
    return state,reward,done    

class Memory():

    def __init__(self, memory_size, batch_size):
        self.buffer = deque (maxlen=memory_size)

        self.batch_size = batch_size

    def add(self, experience):

# Adds a (state, action, reward, state_prime, done) tuple. 
        self.buffer.append(experience)

    def sample(self):

        buffer_size = len(self.buffer)

        index= np.random.choice(np.arange(buffer_size), size=self.batch_size)

        
        batch = [self.buffer[i] for i in index]

        return batch

memory = Memory(memory_size= 1000000 ,batch_size=64  )

def deep_q_network (state_shape, action_size, learning_rate, hidden_neurons):
    state_input = Input((state_shape,), name='frames') 
    actions_input = Input((action_size,), name='mask')

    hidden_1 = Dense(hidden_neurons, activation=tf.keras.activations.relu)(state_input)

    hidden_2 = Dense(hidden_neurons, activation=tf.keras.activations.relu) (hidden_1)
    q_values = Dense(action_size)(hidden_2)

    masked_q_values = Multiply()([q_values, actions_input])

    model= Model(inputs = [state_input, actions_input], outputs=masked_q_values)
    optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

def act(state, training=False):
    random_rate = 1
    random_decay = 0.01
    if training:

# Random actions until enough simulations to train the model.

        if len(memory.buffer) >= memory.batch_size:
            random_rate= random_decay

        if random_rate > np.random.rand(): return random.randint(0, action_size-1)

# If not acting randomly, take action with highest predicted value.

    state_batch = np.expand_dims (state, axis=0)
    predict_mask = np.ones((1, action_size,))
    print(state_batch)
    print(predict_mask)
    action_qs = network.predict([state_batch, predict_mask])
    print(action_qs)
    return np.argmax (action_qs[0])

network = deep_q_network (state_shape, action_size, learning_rate, hidden_neurons)

def update_Q():
    sample = (memory.sample())
    state_mb= sample[0] 
    action_mb = sample[0]
    reward_mb = sample[0]
    state_prime_mb = sample[0]
    done_mb = sample[0]

# Get Q values for state_prime_mb.
   
    predict_mask = np.ones((1, action_size,))
    state_prime = np.expand_dims (state_prime_mb[3], axis=0)
    next_q_mb= network.predict([state_prime, predict_mask])
    next_q_mb= tf.math.reduce_max (next_q_mb, axis=1)
   
# Apply the Bellman Equation

    target_qs = (next_q_mb * memory_gamma) + reward_mb[2]
    target_qs = tf.where(done_mb[4], reward_mb[2], target_qs)
    
# Match training batch to network output

    action_mb = tf.convert_to_tensor (action_mb[1], dtype=tf.int32)
    action_hot = tf.one_hot (action_mb[1], action_size)
    target_mask = tf.multiply(tf.expand_dims (target_qs, -1), action_hot)
    print(action_hot)
    print(state_mb[0])
    state_mb = np.expand_dims(state_mb[0],axis = 0)
    print(state_mb)
    action_hot =  np.expand_dims(action_hot,axis=0)
    print(action_hot)
    return network.train_on_batch([state_mb, action_hot], target_mask)

import sys
import os
from matplotlib.pyplot import show
 
# Import non-standard modules.
import pygame
from pygame.locals import *
from button import Button
from enemy_spawner import EnemySpawner
from random import randrange
import time


pygame.init() 
pygame.display.set_caption("Welcome Day") #The window title

#Import images and define sizes
width, height = 720, 480 
WIZARD_WIDTH , WIZARD_HEIGHT = 78 , 103
LIFE_WIDTH , LIFE_HEIGHT = 21 , 22 
CARA_WIZARD_IMAGE =pygame.image.load(os.path.join('Assets','cara.png')) #character image

CARA_WIZARD=pygame.transform.scale(CARA_WIZARD_IMAGE,(WIZARD_WIDTH,WIZARD_HEIGHT)) #resize

BULLET_FIRE=pygame.image.load(os.path.join('Assets','fireball.png'))
BULLET_FIRE=pygame.transform.scale(BULLET_FIRE, (28,12))
screen = pygame.display.set_mode((width, height))
BULLET_VEL= 7
vel=5
width, height = 720, 480 
MAX_BULLET = 3
bullets=[]
menu=pygame.image.load(os.path.join('Assets','menu.png'))
menu=pygame.transform.scale(menu,(width,height))
background=pygame.image.load(os.path.join('Assets','sky.jpg')) #background image
background=pygame.transform.scale(background, (width,height))
gameover=pygame.image.load(os.path.join('Assets','gameover.jpg'))
gameover=pygame.transform.scale(gameover, (width, height))
pygame.font.init()
pygame.mixer.init()
fireball_sfx = pygame.mixer.Sound(os.path.join('SFX','fireball.mp3'))
font = pygame.font.Font(os.path.join('Fonts', 'FreeSansBold.ttf'), 32)
textX = 10
textY = 10
score = 0
lounge = pygame.mixer.music.load(os.path.join('SFX','lounge.mp3'))

def show_score(x,y, screen):
  
  score_text = font.render("Score: " + str(score), True, (255, 255, 255))
  screen.blit(score_text,(x,y))
  pygame.display.flip()



def update(wizard):
  # Go through events that are passed to the script by the window.
   for event in pygame.event.get():
        if event.type == pygame.QUIT:
          pygame.quit()
          sys.exit()
        if event.type ==pygame.KEYDOWN:
          if event.key == pygame.K_SPACE  and len(bullets)<MAX_BULLET : # event to shot bullets
            bullet= pygame.Rect(wizard.x + wizard.width , wizard.y + wizard.height//2 , 10 , 5)
            bullets.append(bullet)
 
def draw(screen , wizard,bullets, enemy_spawner):
 
  screen.fill((0, 0, 0)) # Fill the screen with black.
  screen.blit(background,(0,0))
  # Redraw screen here.
  screen.blit(CARA_WIZARD , (wizard.x , wizard.y)) # display character

  enemy_spawner.enemy_group.draw(screen)
  for bullet in bullets :
    screen.blit(BULLET_FIRE, bullet ) #display bullets 
  
  # Flip the display so that the things we drew actually show up.
  pygame.display.flip()

# function to handle the character mouvements (Up,Down)
def handle_movement(keys_pressed, wizard):
  if keys_pressed[pygame.K_UP] and wizard.y - vel > 0: #UP
    wizard.y-=vel
  if keys_pressed[pygame.K_DOWN] and wizard.y + vel + wizard.height < height : #DOWN
    wizard.y+=vel

#function to handle the bullet mouvement
def handle_bullet(bullets, wizard, enemy_spawner):
  
  for bullet in bullets:
    bullet.x += BULLET_VEL
    if wizard.colliderect(bullet):
      

      bullets.remove(bullet)
    elif bullet.x> width :
      bullets.remove(bullet)

    global score 
    for enemy in enemy_spawner.enemy_group:
      if bullet.colliderect(enemy):
        enemy.kill()
        score += 1
        pygame.mixer.Sound.play(fireball_sfx)
        bullets.remove(bullet)

def get_font(size): 
  return pygame.font.Font("Fonts/font.ttf", size)



def play():
  # Initialise PyGame.
  global score

  # Set up the clock. This will tick every frame and thus maintain a relatively constant framerate. Hopefully.
  fps = 60.0
  
  fpsClock = pygame.time.Clock()
  
  # Set up the window.

  
  enemy_spawner = EnemySpawner()

  #character
  wizard=pygame.Rect(20 ,100, WIZARD_WIDTH , WIZARD_HEIGHT )
  
  
  # Main game loop.
  dt = 1/fps # dt is the time since last frame.
  run=True
  game_over = False
  while run: # Loop forever!

    fpsClock.tick(fps) # You can update/draw here, I've just moved the code for neatness.
    
    if game_over == False:
      draw(screen , wizard , bullets, enemy_spawner)
      
      
      enemy_spawner.update()
      
      #handeling some events
      update(wizard)
        
      for enemy in enemy_spawner.enemy_group:
        if wizard.colliderect(enemy):
          
          game_over = True
     
      keys_pressed = pygame.key.get_pressed()
      show_score(560,40, screen)
      state = [wizard.y]
      for enemy in enemy_spawner.enemy_group:
        if enemy.rect.x>0:
            state.append(enemy.rect.x)
            state.append(enemy.rect.y)
      length = len(state)  
      for i in range(length,7):
        state.append(-1)
      for bullet in bullets:
        state.append(bullet.x)
        state.append(bullet.y)
      length = len(state)
      for i in range(length,13):
        state.append(-1)
      action= act(state,training = True)
      action = actions[action]  
      n_state,reward,done = step(bullets,enemy_spawner,wizard,action,state)
      exp = [state,action,reward,n_state,done]
      memory.add(exp)
      update_Q()
      handle_bullet(bullets,wizard, enemy_spawner)
    else:
      screen.blit(gameover,(0,0))
      pygame.display.update()
      for event in pygame.event.get():
        if event.type == pygame.QUIT:
          pygame.quit()
          sys.exit()
        if event.type == pygame.KEYDOWN:
          if event.key == pygame.K_RETURN:
            score=0
            play()
      
    

def main():
  
  fps = 60.0
  fpsClock = pygame.time.Clock()
  dt = 1/fps 
  pygame.mixer.music.set_volume(0.1)
  pygame.mixer.music.play()
  
  
  while True :
    fpsClock.tick(fps)
    screen.blit(menu,(0,0))

    PLAY_MOUSE_POS=pygame.mouse.get_pos() # get mouse position
    PLAY_BUTTON = Button(None, pos=(width/2 -3, height/2 +51), 
                            text_input="START", font=get_font(33), base_color="#ffdd00", hovering_color="#E6A2C7")
    PLAY_BUTTON.changeColor(PLAY_MOUSE_POS)
    PLAY_BUTTON.update(screen)
    for event in pygame.event.get():
      if event.type == pygame.QUIT:
        pygame.quit() # Opposite of pygame.init
        sys.exit() 
      if event.type == pygame.MOUSEBUTTONDOWN:
        if PLAY_BUTTON.checkForInput(PLAY_MOUSE_POS):
          for i in range(50000):  
              play()
    pygame.display.update()
 


main()

C:\Users\pc\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


1/1 [==============================] - 0s 75ms/step
tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
[105, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[105  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[0. 1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[110, -1, -1, -1, -1, -1, -1, 105, 156, -1, -1, -1, -1]
[[110  -1  -1  -1  -1  -1  -1 105 156  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
1/1 [==============================] - 0s 41ms/step
tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
[105, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[105  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[0. 1. 0. 0.]]
1/1 [==============================] - 0s 42ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[115, -1, -1, -1, -1, -1, -1, 112, 156, -1, -1, -1, -1]
[[115  -1  -1  -1  -1  -1  -1 112 156  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
1/1 [==============================] - 0s 34ms/step
tf.Tensor([0. 1. 0. 0.],

1/1 [==============================] - 0s 33ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[110, -1, -1, -1, -1, -1, -1, 231, 156, 210, 171, 189, 176]
[[110  -1  -1  -1  -1  -1  -1 231 156 210 171 189 176]]
[[1. 0. 0. 0.]]
1/1 [==============================] - 0s 34ms/step
tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
[120, -1, -1, -1, -1, -1, -1, 315, 156, 294, 171, 273, 176]
[[120  -1  -1  -1  -1  -1  -1 315 156 294 171 273 176]]
[[0. 1. 0. 0.]]
1/1 [==============================] - 0s 43ms/step
tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
[110, -1, -1, -1, -1, -1, -1, 287, 156, 266, 171, 245, 176]
[[110  -1  -1  -1  -1  -1  -1 287 156 266 171 245 176]]
[[0. 1. 0. 0.]]
1/1 [==============================] - 0s 31ms/step
tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
[110, -1, -1, -1, -1, -1, -1, 175, 156, 154, 171, 133, 176]
[[110  -1  -1  -1  -1  -1  -1 175 156 154 171 133 176]]
[[0. 1. 0. 0.]]
1/1 [==============================] - 0s 33ms/step
tf.T

1/1 [==============================] - 0s 33ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[110, -1, -1, -1, -1, -1, -1, 245, 156, 224, 171, 203, 176]
[[110  -1  -1  -1  -1  -1  -1 245 156 224 171 203 176]]
[[1. 0. 0. 0.]]
[[140 450 302  -1  -1  -1  -1 574 156 553 171 532 176]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 34ms/step
[[ 98.54348   91.09706  -24.343706  43.391647]]
1/1 [==============================] - 0s 33ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[125, -1, -1, -1, -1, -1, -1, 126, 156, 105, 171, -1, -1]
[[125  -1  -1  -1  -1  -1  -1 126 156 105 171  -1  -1]]
[[1. 0. 0. 0.]]
[[145 436 302  -1  -1  -1  -1 581 156 560 171 539 176]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 34ms/step
[[109.055916  92.4686   -23.879349  43.616673]]
1/1 [==============================] - 0s 33ms/step
tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
[115, -1, -1, -1, -1, -1, -1, 364, 156, 343, 171, 322, 176]
[[115  -1  -1  -1  -1 

1/1 [==============================] - 0s 35ms/step
[[165.16296  231.90291  -32.866535  48.21862 ]]
1/1 [==============================] - 0s 31ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[135, 282, 302, 758, 306, -1, -1, 658, 156, 637, 171, 616, 176]
[[135 282 302 758 306  -1  -1 658 156 637 171 616 176]]
[[1. 0. 0. 0.]]
[[125 156 302 632 306  -1  -1 693 171 679 176  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 32ms/step
[[255.03728  189.75883  -33.30773   31.487677]]
1/1 [==============================] - 0s 32ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[135, 478, 302, -1, -1, -1, -1, 560, 156, 539, 171, 518, 176]
[[135 478 302  -1  -1  -1  -1 560 156 539 171 518 176]]
[[1. 0. 0. 0.]]
[[130 142 302 618 306  -1  -1 700 171 686 176  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 35ms/step
[[216.73091  186.41092  -33.57353   33.047142]]
1/1 [==============================] - 0s 35ms/step
tf.Tensor([0. 1. 0. 0.], s

...

1/1 [==============================] - 0s 58ms/step
tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
[10, 696, 352, -1, -1, -1, -1, 154, 76, 154, 71, -1, -1]
[[ 10 696 352  -1  -1  -1  -1 154  76 154  71  -1  -1]]
[[0. 1. 0. 0.]]
[[115 611  31  -1  -1  -1  -1 224 191 182 211 175 206]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 31ms/step
[[25.636692   32.409145    0.11249113 23.476046  ]]
1/1 [==============================] - 0s 28ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[165, 728, 31, -1, -1, -1, -1, 133, 191, -1, -1, -1, -1]
[[165 728  31  -1  -1  -1  -1 133 191  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[110 602  31  -1  -1  -1  -1 231 191 189 211 182 206]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 30ms/step
[[29.034985  33.31418    0.5614114 22.421017 ]]
1/1 [==============================] - 0s 23ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[5, 450, 121, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[  5 450 121  -1  -1  -1  -

1/1 [==============================] - 0s 36ms/step
[[45.70695  63.668816  9.699448 20.191032]]
1/1 [==============================] - 0s 60ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[25, 638, 35, -1, -1, -1, -1, 553, 126, 525, 106, 336, 101]
[[ 25 638  35  -1  -1  -1  -1 553 126 525 106 336 101]]
[[1. 0. 0. 0.]]
[[ 10 422  31 745 171  -1  -1 371 191 329 211 322 206]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 31ms/step
[[49.43621   64.43532   10.3339205 18.979836 ]]
1/1 [==============================] - 0s 31ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[95, -1, -1, -1, -1, -1, -1, 105, 116, -1, -1, -1, -1]
[[ 95  -1  -1  -1  -1  -1  -1 105 116  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[  5 413  31 734 171  -1  -1 378 191 336 211 329 206]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 34ms/step
[[49.595383 64.03571  10.359272 18.777905]]
1/1 [==============================] - 0s 31ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=fl

1/1 [==============================] - 0s 27ms/step
[[56.15871  55.030743 10.791073 14.68561 ]]
1/1 [==============================] - 0s 40ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[80, 500, 116, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[ 80 500 116  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[ 10 233  31  -1  -1  -1  -1 469 211 469 206  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 43ms/step
[[38.06374   33.310978  -2.0290513 11.344811 ]]
1/1 [==============================] - 0s 48ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[245, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[245  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[ 15 224  31  -1  -1  -1  -1 476 211 476 206  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 53ms/step
[[37.860138  33.220543  -2.4019806 11.542961 ]]
1/1 [==============================] - 0s 44ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=floa

1/1 [==============================] - 0s 33ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[5, 371, 140, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[  5 371 140  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[115  -1  -1  -1  -1  -1  -1 609 211 609 206  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 25ms/step
[[37.342934 32.14489  -7.104389 20.633415]]
1/1 [==============================] - 0s 35ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[5, 59, 191, 470, 40, -1, -1, -1, -1, -1, -1, -1, -1]
[[  5  59 191 470  40  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[120  -1  -1  -1  -1  -1  -1 616 211 616 206  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 27ms/step
[[38.0091    32.6051    -7.2388372 20.70132  ]]
1/1 [==============================] - 0s 24ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[5, 40, 406, 638, 133, -1, -1, -1, -1, -1, -1, -1, -1]
[[  5  40 406 638 133  -1  -1  -1  -1  -1

[[125  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 67ms/step
[[10.14973    8.922453  -1.9327147  1.8439891]]
1/1 [==============================] - 0s 22ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[20, 90, 171, 501, 233, -1, -1, 196, 56, -1, -1, -1, -1]
[[ 20  90 171 501 233  -1  -1 196  56  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[130  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 25ms/step
[[10.396545   9.260575  -1.9646833  1.8654097]]
1/1 [==============================] - 0s 40ms/step
tf.Tensor([1. 0. 0. 0.], shape=(4,), dtype=float32)
[5, 358, 27, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[  5 358  27  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 0. 0. 0.]]
[[135  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]
[[1. 1. 1. 1.]]
1/1 [==============================] - 0s 54ms/step
[[10.435922   9.582582  -2.0470674  1.9055588]]
1/1 [=================

In [1]:
print(state_prime_mb[0])

NameError: name 'state_prime_mb' is not defined

In [5]:
index= np.random.choice(np.arange(100), size=64)
index


array([10, 40, 88, 25, 43, 82, 37, 59, 82, 58, 84, 59,  9, 53, 82, 41, 90,
       13, 42, 79, 62, 62, 86, 88, 20,  1, 13, 10,  4, 86, 18, 34, 42, 58,
       76, 19, 56, 55, 14, 31, 21, 27, 90, 27, 61, 59,  2, 55, 64, 12, 84,
       49, 78, 75, 98, 20,  7, 83, 37, 87,  4, 53, 57, 58])

In [6]:
memory.sample()

[[[75, 800, 196, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  (-1, 0),
  1,
  [75, 800, 196, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  False],
 [[20, 624, 153, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  (-1, 0),
  1,
  [20, 624, 153, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  False],
 [[80, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  (-1, 0),
  1,
  [80, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  False],
 [[40, 660, 205, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  (-1, 0),
  1,
  [40, 660, 205, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  False],
 [[60, 650, 196, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  (1, 0),
  1,
  [60, 650, 196, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  False],
 [[105, 430, 205, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  (1, 0),
  1,
  [105, 430, 205, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  False],
 [[55, 156, 227, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  (-1, 0),
  1,
  [55, 156, 227, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  False],
 [[65, 160

In [7]:
f = memory.sample()
r = f[0]
r

[[210, 464, 105, -1, -1, -1, -1, 420, 221, -1, -1, -1, -1],
 (1, 0),
 1,
 [210, 464, 105, -1, -1, -1, -1, 420, 221, -1, -1, -1, -1],
 False]